# Functions and import

In [4]:
import pandas as pd
import os
import random
import numpy as np
import time
import re

import os
import glob

from Environment.gens.TA_Gen import TAStreamer
from Environment.envs.indicator_1 import Indicator_1

from Agent.duelling_dqn import DDDQNAgent

In [2]:
def World(filename=None,
        train_test = 'train',
        episodes=10,
        train_test_split = 0.75,
        trading_fee = .0001,
        time_fee = .001,
        memory_size = 3000,
        gamma = 0.96,
        epsilon_min = 0.01,
        batch_size = 64,
        train_interval = 10,
        learning_rate = 0.001,
        render_show=False,
        display=False,
        save_results=False
):
    start = time.time()

    generator = TAStreamer(filename=filename, mode='train', split=train_test_split)
    episode_length = round(int(len(pd.read_csv(filename))*train_test_split), -1)

    environment = Indicator_1(data_generator=generator,
                              trading_fee=trading_fee,
                              time_fee=time_fee,
                              episode_length=episode_length)
    action_size = len(Indicator_1._actions)

    state = environment.reset()

    state_size = len(state)


    try:
        symbol = re.findall(r'Data\\([^_]+)',filename)[0]
    except:
        symbol = ""

    agent = DDDQNAgent(state_size=state_size,
                     action_size=action_size,
                     memory_size=memory_size,
                     episodes=episodes,
                     episode_length=episode_length,
                     train_interval=train_interval,
                     gamma=gamma,
                     learning_rate=learning_rate,
                     batch_size=batch_size,
                     epsilon_min=epsilon_min,
                     train_test=train_test,
                     symbol=symbol)

    # Warming up the agent
    if (train_test == 'train'):
        for _ in range(memory_size):
            action = agent.act(state)
            next_state, reward, done, _ = environment.step(action)
            agent.observe(state, action, reward, next_state, done, warming_up=True)
        if display:
            print('completed mem allocation: ', time.time() - start)

    # Training the agent
    loss_list=[]
    val_loss_list=[]
    reward_list=[]
    epsilon_list=[]
    metrics_df=None
    if train_test == "train":
        best_loss = 9999
        best_reward = 0
        for ep in range(episodes):
            ms = time.time()
            state = environment.reset()
            rew = 0
            loss_list_temp = []
            val_loss_list_temp = []

            for _ in range(episode_length):
                action = agent.act(state)
                next_state, reward, done, _ = environment.step(action)
                loss = agent.observe(state, action, reward, next_state,
                                     done)  # loss would be none if the episode length is not % by 10
                state = next_state
                rew += reward
                if(loss):
                    loss_list_temp.append(round(loss.history["loss"][0],3))
                    val_loss_list_temp.append(round(loss.history["val_loss"][0],3))

            if display:
                print("Ep:" + str(ep)
                      + "| rew:" + str(round(rew, 2))
                      + "| eps:" + str(round(agent.epsilon, 2))
                      + "| loss:" + str(round(loss.history["loss"][0], 4))
                      + "| runtime:" + str(time.time() - ms))
                print("Loss=", str(np.mean(loss_list_temp)), " Val_Loss=", str(np.mean(val_loss_list_temp)))

            loss_list.append(np.mean(loss_list_temp))
            val_loss_list.append(np.mean(val_loss_list_temp))
            reward_list.append(rew)
            epsilon_list.append(round(agent.epsilon, 2))

        agent.save_model()

        metrics_df=pd.DataFrame({'loss':loss_list,'val_loss':val_loss_list,'reward':reward_list,'epsilon':epsilon_list})

        if save_results:
            metrics_df.to_csv('./Results/perf_metrics.csv')

    if(train_test=='test'):
        agent.load_model()

    generator = TAStreamer(filename=filename, mode='test', split=train_test_split)
    environment = Indicator_1(data_generator=generator,
                              trading_fee=trading_fee,
                              time_fee=time_fee,
                              episode_length=episode_length,)

    done = False
    state = environment.reset()
    q_values_list=[]
    state_list=[]
    action_list=[]
    reward_list=[]
    trade_list=[]

    while not done:
        action, q_values = agent.act(state, test=True)
        state, reward, done, info = environment.step(action)
        if 'status' in info and info['status'] == 'Closed plot':
            done = True
        else:
            reward_list.append(reward)

            calc_returns=environment.return_calc(render_show)
            if calc_returns:
                trade_list.append(calc_returns)

            if(render_show):
                environment.render()


        q_values_list.append(q_values)
        state_list.append(state)
        action_list.append(action)

    print('Reward = %.2f' % sum(reward_list))

    trades_df=pd.DataFrame(trade_list)
    action_policy_df = pd.DataFrame({'q_values':q_values_list,'state':state_list,'action':action_list})

    if save_results:
        trades_df.to_csv(r'./Results/trade_list.csv')
        action_policy_df.to_pickle(r'./Results/action_policy.pkl')

    if display:
        print("All done:", str(time.time() - start))

    return({"metrics_df":metrics_df,
            "trades_df":trades_df,
            "action_policy_df":action_policy_df,
            "reward_list":reward_list})

## get results 

In [6]:
#stocks_csv=glob.glob(r"./Data/*.csv")
stocks_csv=glob.glob(r"./Data/aprova/*.csv")
stocks_csv.sort()
len(stocks_csv)
#symbol_path=dict(zip([re.findall(r'Data\\([^_]+)',x)[0] for x in stocks_csv],stocks_csv))
# stocks_csv

1

In [7]:
train_test_split=0.75
results_dict={}

for i,stocks in enumerate(stocks_csv):
    print(i)
    print(stocks)
    start = time.time()
    results=World(filename=stocks,train_test_split=train_test_split,episodes=100)
    end = time.time()
    print('Time Taken = ',end-start)
    df=pd.read_csv(stocks)
    df=df.iloc[int(train_test_split*len(df)):,:]
    returns=df.close.pct_change()
    perf_metrics = sharpe_calc(results['trades_df'].copy())

    results['buy_and_hold_sharpe']=np.mean(returns)/np.std(returns)
    results['strategy_sharpe']=perf_metrics['strategy_sharpe']
    results['num_trades']=perf_metrics['num_trades']
    results['position_df']=perf_metrics['position_df']

    results_dict[df.Name.iloc[0]]=results
    gc.collect()

pickle.dump(results_dict, open("./Results/results_revised.p","wb"))

0
./Data/aprova/prova.csv


/home/klop/.local/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


AttributeError: 'Series' object has no attribute 'as_matrix'

# Analytics

In [ ]:
results_dict = pickle.load(open(r"./Results/results_revised.p", "rb"))

In [ ]:
#compare with others
results_df=pd.DataFrame({'Symbols':list(results_dict.keys()),
'Buy and Hold Sharpe':[x['buy_and_hold_sharpe'] for x in list(results_dict.values())],
'Strategy Sharpe':[x['strategy_sharpe'] for x in list(results_dict.values())],
'Number of Trades':[x['num_trades'] for x in list(results_dict.values())],
'Buy and Hold Total Return':[x['buy_n_hold_tot_return'] for x in list(results_dict.values())],
'Strategy Total Return':[x['strategy_tot_return'] for x in list(results_dict.values())],
})
results_df_filt=results_df.dropna()
results_df_filt.head()

In [ ]:
#save as csv
results_df_filt.to_csv("Trade Analytics Results.csv")

In [ ]:
#get chart 
test=results_df_filt[results_df_filt['Number of Trades']>2]
x0 = np.random.randn(500)
x1 = np.random.randn(500)+1

trace1 = Histogram(
    x=test["Strategy Sharpe"],
    opacity=0.75,
    name='Buy_n_Hold'
)
trace2 = Histogram(
    x=test['Buy and Hold Sharpe'],
    opacity=0.75,
    name='DDDQN'
)

data = [trace1, trace2]
layout = Layout(title='Risk Adjusted return Histogram',barmode='overlay')
fig = Figure(data=data, layout=layout)

py.iplot(fig, filename='overlaid histogram')